# 🤖 Anotação sem LLM Hacking

## 1) Setup e Configuração

In [1]:
import sys
from loguru import logger
from src.api.schemas.experiment import ExperimentRequest

logger.remove()
logger.add(
    sys.stdout,
    format="<green>{time:HH:mm:ss}</green> | <level>{level: <8}</level> | <level>{message}</level>",
    level="INFO"
)

logger.success("✓ Setup completo")

14:12:39 | SUCCESS  | ✓ Setup completo


In [3]:
import json
from pathlib import Path

experiment = "time_experiment_local"

# Load JSON from file
config_path = Path(f"../experiments/{experiment}.json")

with open(config_path, "r") as f:
    config_dict = json.load(f)

# Instantiate Pydantic model
EXPERIMENT_CONFIG = ExperimentRequest(**config_dict)

### - Modelos e prompt

In [4]:
from src.api.services.prompt_factory import get_prompt_template

DEFAULT_MODELS = EXPERIMENT_CONFIG.models
PROMPT_TEMPLATE = EXPERIMENT_CONFIG.prompt_type

PROMPT_TEMPLATE  = get_prompt_template(
    EXPERIMENT_CONFIG.prompt_type,
    EXPERIMENT_CONFIG.custom_prompt,
)

### - Configurações de anotação

In [5]:
annotation_cfg = EXPERIMENT_CONFIG.annotation

num_repetitions = annotation_cfg.num_repetitions_per_llm
use_alternative_params = annotation_cfg.use_alternative_params

model_strategy = annotation_cfg.model_strategy
rep_strategy = annotation_cfg.rep_strategy

### - Configurações de dataset

In [7]:
dataset_cfg = EXPERIMENT_CONFIG.dataset_config

dataset_split = dataset_cfg.split
combine_splits = dataset_cfg.combine_splits
sample_size = dataset_cfg.sample_size
random_state = dataset_cfg.random_state

### - Configurações de cache

In [8]:
cache_cfg = EXPERIMENT_CONFIG.cache

cache_enabled = cache_cfg.enabled
cache_dir = cache_cfg.dir

### - Resultados

In [9]:
results_cfg = EXPERIMENT_CONFIG.results

save_intermediate = results_cfg.save_intermediate
intermediate = results_cfg.intermediate
results_dir = results_cfg.dir

## 2) Carregar Dataset

In [10]:
from src.utils.data_loader import load_hf_dataset, load_hf_dataset_as_dataframe, list_available_datasets, get_dataset_info


logger.info("Datasets disponíveis:")
for dataset in list_available_datasets():
    logger.info(f"  - {dataset}")

14:12:40 | Datasets disponíveis:
14:12:40 |   - agnews
14:12:40 |   - mpqa
14:12:40 |   - webkb
14:12:40 |   - ohsumed
14:12:40 |   - acm
14:12:40 |   - yelp_2013
14:12:40 |   - dblp
14:12:40 |   - books
14:12:40 |   - reut90
14:12:40 |   - wos11967
14:12:40 |   - twitter
14:12:40 |   - trec
14:12:40 |   - wos5736
14:12:40 |   - sst1
14:12:40 |   - pang_movie
14:12:40 |   - movie_review
14:12:40 |   - vader_movie
14:12:40 |   - subj
14:12:40 |   - sst2
14:12:40 |   - yelp_reviews
14:12:40 |   - 20ng
14:12:40 |   - medline


In [11]:
dataset_name = "sst1"  # Ajuste conforme necessário

texts, categories, ground_truth = load_hf_dataset(
    dataset_name=dataset_name, 
    cache_dir=cache_dir,
    dataset_global_config=dataset_cfg
)

logger.info(f"Textos: {len(texts)}")
logger.info(f"Categorias: {categories}")
logger.info(f"Ground truth: {'Sim' if ground_truth else 'Não'}")

14:12:40 | Carregando dataset: sst1
14:12:40 | Combinando splits: ['train', 'test']
14:12:47 |   ✓ train: 106695 exemplos
14:12:54 |   ✓ test: 11855 exemplos
14:12:54 | Total combinado: 118550 exemplos
14:12:54 | Categorias extraídas automaticamente: [0, 1, 2, 3, 4]
14:12:54 | Amostra reduzida para 20 exemplos (seed=42)
14:12:54 | Coluna de texto: text
14:12:54 | Ground truth carregado da coluna 'label'
14:12:54 | Textos: 20
14:12:54 | Categorias: [0, 1, 2, 3, 4]
14:12:54 | Ground truth: Sim


In [12]:
# Visualizar amostra
logger.info("Amostra dos textos:")
for i, text in enumerate(texts[:3]):
    logger.info(f"{i+1}. {text[:100]}...")
    if ground_truth:
        logger.info(f"   Label: {ground_truth[i]}")

14:12:54 | Amostra dos textos:
14:12:54 | 1. impostor has a handful of thrilling moments and a couple of good performances , but the movie does n...
14:12:54 |    Label: 1
14:12:54 | 2. the acting in pauline and paulette is good all round , but what really sets the film apart is debrau...
14:12:54 |    Label: 4
14:12:54 | 3. mr. polanski is in his element here : alone , abandoned , but still consoled by his art , which is m...
14:12:54 |    Label: 3


In [13]:
df, categories = load_hf_dataset_as_dataframe(
    dataset_name=dataset_name,
    cache_dir=cache_dir,
    dataset_global_config=dataset_cfg
)

14:12:54 | Carregando dataset: sst1
14:12:54 | Combinando splits: ['train', 'test']
14:13:00 |   ✓ train: 106695 exemplos
14:13:07 |   ✓ test: 11855 exemplos
14:13:07 | Total combinado: 118550 exemplos
14:13:07 | Categorias extraídas automaticamente: [0, 1, 2, 3, 4]
14:13:07 | Amostra reduzida para 20 exemplos (seed=42)
14:13:07 | Coluna de texto: text
14:13:07 | Ground truth carregado da coluna 'label'
14:13:07 | DataFrame criado com 20 linhas


## 3) Configurar Modelos LLM

### Opção A: Usar apenas parâmetros padrão (temp=0)

In [14]:
from src.llm_annotation_system.annotation.llm_annotator import LLMAnnotator

annotator = LLMAnnotator(
    dataset_name=dataset_name,
    models=DEFAULT_MODELS,
    categories=categories,
    cache_dir=cache_dir,
    results_dir=results_dir,
    prompt_template=PROMPT_TEMPLATE,
    use_langchain_cache=cache_enabled,
    use_alternative_params=use_alternative_params
)

logger.success(f"✓ Annotator inicializado com {len(annotator.models)} modelos")

14:13:09 | Cache carregado: 17361 entradas
14:13:09 | Cache LangChain DESATIVADO explicitamente
14:13:09 | Template do prompt preparado


Ollama params: {}
Ollama params: {}
Ollama params: {}
Ollama params: {}


14:13:11 | LLMAnnotator inicializado
14:13:11 | Modelos: 5 | Categorias: 5
14:13:11 | ✓ Annotator inicializado com 5 modelos


## 4) Executar Anotação

In [15]:
df

,text,label,label_description
0,impostor has a handful of thrilling moments an...,1,negative sentiment
1,the acting in pauline and paulette is good all...,4,very positive sentiment
2,"mr. polanski is in his element here : alone , ...",3,positive sentiment
3,a cop story that understands the medium amazin...,3,positive sentiment
4,the most horrific movie experience i 've had s...,0,very negative sentiment
5,"massoud 's story is an epic , but also a trage...",2,neutral sentiment
6,the actors do n't inhabit their roles -- they ...,0,very negative sentiment
7,more of a career curio than a major work .\n,2,neutral sentiment
8,"there 's a lot of good material here , but the...",2,neutral sentiment
9,if you saw benigni 's pinocchio at a public pa...,1,negative sentiment


In [16]:
annotator.annotation_engine.template

'You are an expert data annotator with extensive experience in text classification tasks.\n\nYour task is to classify the following text into one of the predefined categories with high precision.\n\n**Instructions:**\n1. Read the text carefully and understand its context\n2. Consider the nuances and implicit meanings\n3. Select the most appropriate category based on the content\n4. Be consistent with your classification criteria\n5. If the text is ambiguous, choose the most likely category based on dominant features\n\n**Available Categories:**\n- 0: very negative sentiment\n- 1: negative sentiment\n- 2: neutral sentiment\n- 3: positive sentiment\n- 4: very positive sentiment\n\n**Text to classify (Opinion / Sentiment):**\n{text}\n\n**Important Guidelines:**\n- Provide ONLY the category number as your response\n- Do not include explanations\n- Be objective and avoid bias\n- Consider edge cases carefully\n- Maintain consistency across similar texts\n\n**Your classification to the opinio

### - Testar anotação única

In [18]:
# Parâmetros de anotação
text = texts[0]
model = "deepseek-r1-distill-llama-8b"

total_annotation = len(text) * num_repetitions
logger.warning(f"  Modelo: {model}")
logger.warning(f"  Texto: {len(text)}")
logger.warning(f"  Repetições: {num_repetitions}")

annotations = await annotator.annotate_single(
        text=text,
        model=model,
        num_repetitions=3,
        use_cache=cache_enabled,
        rep_strategy=rep_strategy
    )

logger.success("✓ Anotação completa")

annotations

14:13:11 |   Modelo: deepseek-r1-distill-llama-8b
14:13:11 |   Texto: 115
14:13:11 |   Repetições: 1
14:13:18 | ✓ Anotação completa


[2, 1, 1]

### - Anotando dataset completo

In [ ]:
df_annotations = await annotator.annotate_dataset(
    texts=texts,
    num_repetitions=num_repetitions,
    use_cache=cache_enabled,
    save_intermediate=save_intermediate,
    intermediate=intermediate,
    model_strategy=model_strategy,
    rep_strategy=rep_strategy
)

df_annotations["ground_truth"] = ground_truth if ground_truth else None

logger.success("✓ Anotações completas")
df_annotations

14:13:18 | Iniciando anotação
14:13:18 | Textos: 20 | Modelos: 5 | Repetições: 1
14:13:18 | Total de anotações: 100
Anotando: 100%|██████████| 20/20 [08:32<00:00, 25.64s/it]
14:21:50 | ✓ Anotações completas


,text_id,text,deepseek-r1-distill-llama-8b_rep1,deepseek-r1-distill-llama-8b_consensus,deepseek-r1-distill-llama-8b_consensus_score,deepseek-r1-distill-llama-8b_annotation_time_sec,qwen3-8b_rep1,qwen3-8b_consensus,qwen3-8b_consensus_score,qwen3-8b_annotation_time_sec,...,llama2-7b_consensus_score,llama2-7b_annotation_time_sec,mistral-7b_rep1,mistral-7b_consensus,mistral-7b_consensus_score,mistral-7b_annotation_time_sec,llama3.1-8b_rep1,llama3.1-8b_consensus,llama3.1-8b_consensus_score,llama3.1-8b_annotation_time_sec
0,0,impostor has a handful of thrilling moments an...,1,1,1.0,3.435346,1,1,1.0,10.724097,...,1.0,3.956189,1,1,1.0,3.990847,2,2,1.0,4.549660
1,1,the acting in pauline and paulette is good all...,3,3,1.0,4.176737,3,3,1.0,10.415090,...,1.0,3.204977,3,3,1.0,4.058478,2,2,1.0,4.841386
2,2,"mr. polanski is in his element here : alone , ...",3,3,1.0,5.978358,3,3,1.0,15.906592,...,1.0,3.941715,3,3,1.0,2.791691,2,2,1.0,4.139649
3,3,a cop story that understands the medium amazin...,4,4,1.0,3.937454,4,4,1.0,11.799683,...,1.0,4.735499,3,3,1.0,4.011998,3,3,1.0,4.119874
4,4,the most horrific movie experience i 've had s...,0,0,1.0,7.080560,0,0,1.0,9.694489,...,1.0,3.476061,1,1,1.0,2.779737,1,1,1.0,4.093793
5,5,"massoud 's story is an epic , but also a trage...",1,1,1.0,5.691549,2,2,1.0,16.106826,...,1.0,3.444676,3,3,1.0,2.773523,2,2,1.0,4.119426
6,6,the actors do n't inhabit their roles -- they ...,1,1,1.0,5.150180,0,0,1.0,11.109907,...,1.0,3.463867,1,1,1.0,2.815481,1,1,1.0,3.839775
7,7,more of a career curio than a major work .\n,1,1,1.0,4.433974,1,1,1.0,10.562480,...,1.0,2.932793,1,1,1.0,2.054406,2,2,1.0,2.985290
8,8,"there 's a lot of good material here , but the...",3,3,1.0,4.640043,1,1,1.0,22.385393,...,1.0,2.186939,1,1,1.0,0.368188,2,2,1.0,2.970219
9,9,if you saw benigni 's pinocchio at a public pa...,1,1,1.0,6.402905,0,0,1.0,15.425891,...,1.0,2.182142,0,0,1.0,0.358037,1,1,1.0,2.961752


## 5) Salvando resultados

### - Anotações

In [ ]:
from datetime import date 

date_path = date.today()
date_path = date_path.strftime("%Y-%m-%d")
experiment_id = f"{date_path}"

output_dir = annotator.results_dir or f"results/{experiment_id}"
output_dir = Path(output_dir).joinpath(experiment_id)
output_dir.mkdir(parents=True, exist_ok=True)
df_annotations.to_csv(
    Path(output_dir) / "annotations.csv",
    index=False,
)

### - Métricas

In [ ]:
df_metrics = annotator.evaluate_model_metrics(
    df_annotations,
    ground_truth_col="ground_truth",
    output_csv=True,
    output_dir=output_dir
)